In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Text Generation Demo\n",
    "\n",
    "This notebook demonstrates how to use the text generation models implemented in this project. It includes loading the models, preprocessing input data, and generating text samples."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "load-models"
   },
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import torch\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from utils.text_preprocessing import TextPreprocessor\n",
    "from models.lstm_model import LSTMTextGenerator, TextDataset, train_lstm_model\n",
    "from models.gpt_model import GPTTextGenerator\n",
    "from torch.utils.data import DataLoader\n",
    "\n",
    "# Set style\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "\n",
    "print(\"Libraries imported successfully!\")\n",
    "print(f\"PyTorch version: {torch.__version__}\")\n",
    "print(f\"CUDA available: {torch.cuda.is_available()}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "preprocess-data"
   },
   "outputs": [],
   "source": [
    "# Load sample data\n",
    "def load_sample_data():\n",
    "    sample_texts = [\n",
    "        \"Technology is rapidly evolving and changing our daily lives. Artificial intelligence and machine learning are becoming integral parts of modern society. Smart devices connect us globally while automation increases efficiency in various industries.\",\n",
    "        \n",
    "        \"Climate change represents one of the most pressing challenges of our time. Rising temperatures affect weather patterns worldwide. Sustainable energy solutions and environmental conservation efforts are crucial for future generations.\",\n",
    "        \n",
    "        \"Space exploration continues to fascinate humanity and drive scientific advancement. Recent missions to Mars have provided valuable insights about our neighboring planet. Private companies are now contributing significantly to space research and development.\",\n",
    "        \n",
    "        \"Education systems worldwide are adapting to digital transformation. Online learning platforms provide accessible education to students globally. Interactive technologies enhance traditional teaching methods and improve learning outcomes.\",\n",
    "        \n",
    "        \"Healthcare innovation saves lives and improves quality of life for millions. Medical research leads to breakthrough treatments for various diseases. Personalized medicine and genetic therapies represent the future of healthcare.\"\n",
    "    ]\n",
    "    return sample_texts\n",
    "\n",
    "texts = load_sample_data()\n",
    "\n",
    "print(\"Sample Data Loaded:\")\n",
    "print(f\"Number of texts: {len(texts)}\")\n",
    "print(f\"Average text length: {np.mean([len(text) for text in texts]):.1f} characters\")\n",
    "\n",
    "# Display first text\n",
    "print(f\"\\nFirst text sample:\\n{texts[0]}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "generate-text"
   },
   "outputs": [],
   "source": [
    "# Initialize preprocessor\n",
    "preprocessor = TextPreprocessor()\n",
    "\n",
    "# Build vocabulary\n",
    "vocab = preprocessor.build_vocabulary(texts, min_freq=1)\n",
    "\n",
    "print(f\"Vocabulary size: {len(vocab)}\")\n",
    "print(f\"Sample vocabulary items: {list(vocab.items())[:10]}\")\n",
    "\n",
    "# Create sequences\n",
    "sequences = preprocessor.create_sequences(texts, sequence_length=15)\n",
    "print(f\"Number of training sequences: {len(sequences)}\")\n",
    "\n",
    "# Show sample sequence\n",
    "sample_seq = sequences[0]\n",
    "print(f\"\\nSample sequence (indices): {sample_seq}\")\n",
    "print(f\"Sample sequence (words): {preprocessor.sequence_to_text(sample_seq)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create dataset and dataloader\n",
    "dataset = TextDataset(sequences, sequence_length=15)\n",
    "dataloader = DataLoader(dataset, batch_size=16, shuffle=True)\n",
    "\n",
    "# Initialize LSTM model\n",
    "lstm_model = LSTMTextGenerator(\n",
    "    vocab_size=preprocessor.vocab_size,\n",
    "    embedding_dim=64,\n",
    "    hidden_dim=128,\n",
    "    num_layers=2,\n",
    "    dropout=0.2\n",
    ")\n",
    "\n",
    "print(f\"LSTM Model Architecture:\")\n",
    "print(lstm_model)\n",
    "\n",
    "# Train the model\n",
    "print(\"\\nTraining LSTM model...\")\n",
    "losses = train_lstm_model(lstm_model, dataloader, num_epochs=10, learning_rate=0.01)\n",
    "\n",
    "# Plot training loss\n",
    "plt.figure(figsize=(12, 6))\n",
    "plt.plot(losses, marker='o', linewidth=2, markersize=6)\n",
    "plt.title('LSTM Training Loss Over Time', fontsize=16, fontweight='bold')\n",
    "plt.xlabel('Epoch', fontsize=12)\n",
    "plt.ylabel('Loss', fontsize=12)\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f\"Final training loss: {losses[-1]:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate text with different prompts\n",
    "prompts = [\n",
    "    \"Technology\",\n",
    "    \"Climate change\",\n",
    "    \"Space\",\n",
    "    \"Education\",\n",
    "    \"Healthcare innovation\"\n",
    "]\n",
    "\n",
    "print(\"LSTM Generated Text:\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "for i, prompt in enumerate(prompts, 1):\n",
    "    generated = lstm_model.generate_text(\n",
    "        preprocessor, \n",
    "        start_text=prompt, \n",
    "        max_length=40,\n",
    "        temperature=0.8\n",
    "    )\n",
    "    \n",
    "    print(f\"{i}. Prompt: '{prompt}'\")\n",
    "    print(f\"   Generated: {generated}\")\n",
    "    print(\"-\" * 50)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize GPT model\n",
    "print(\"Loading GPT-2 model...\")\n",
    "gpt_generator = GPTTextGenerator('gpt2')\n",
    "\n",
    "# Generate text with GPT\n",
    "print(\"\\nGPT-2 Generated Text:\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "gpt_prompts = [\n",
    "    \"Technology is revolutionizing\",\n",
    "    \"Climate change impacts our\",\n",
    "    \"Space exploration reveals new\",\n",
    "    \"Modern education systems are\",\n",
    "    \"Healthcare innovations provide\"\n",
    "]\n",
    "\n",
    "for i, prompt in enumerate(gpt_prompts, 1):\n",
    "    generated_texts = gpt_generator.generate_text(\n",
    "        prompt=prompt,\n",
    "        max_length=60,\n",
    "        temperature=0.7,\n",
    "        num_return_sequences=1\n",
    "    )\n",
    "    \n",
    "    print(f\"{i}. Prompt: '{prompt}'\")\n",
    "    print(f\"   Generated: {generated_texts[0]}\")\n",
    "    print(\"-\" * 50)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compare both models on the same prompts\n",
    "comparison_prompts = [\"Technology\", \"Climate\", \"Space\"]\n",
    "\n",
    "print(\"Model Comparison:\")\n",
    "print(\"=\" * 60)\n",
    "\n",
    "for prompt in comparison_prompts:\n",
    "    print(f\"\\nPrompt: '{prompt}'\")\n",
    "    print(\"-\" * 30)\n",
    "    \n",
    "    # LSTM generation\n",
    "    lstm_generated = lstm_model.generate_text(\n",
    "        preprocessor, \n",
    "        start_text=prompt, \n",
    "        max_length=30,\n",
    "        temperature=0.8\n",
    "    )\n",
    "    \n",
    "    # GPT generation\n",
    "    gpt_generated = gpt_generator.generate_text(\n",
    "        prompt=prompt,\n",
    "        max_length=50,\n",
    "        temperature=0.8\n",
    "    )[0]\n",
    "    \n",
    "    print(f\"LSTM: {lstm_generated}\")\n",
    "    print(f\"GPT:  {gpt_generated}\")\n",
    "    print(\"=\" * 60)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def interactive_text_generation():\n",
    "    \"\"\"Interactive function for text generation\"\"\"\n",
    "    print(\"Interactive Text Generation\")\n",
    "    print(\"Enter your prompts below (type 'stop' to end)\")\n",
    "    \n",
    "    while True:\n",
    "        user_prompt = input(\"\\nEnter your prompt: \").strip()\n",
    "        \n",
    "        if user_prompt.lower() == 'stop':\n",
    "            break\n",
    "            \n",
    "        if not user_prompt:\n",
    "            continue\n",
    "            \n",
    "        print(\"\\nChoose model: 1) LSTM  2) GPT  3) Both\")\n",
    "        model_choice = input(\"Enter choice (1, 2, or 3): \").strip()\n",
    "        \n",
    "        if model_choice in ['1', '3']:\n",
    "            lstm_result = lstm_model.generate_text(\n",
    "                preprocessor, \n",
    "                start_text=user_prompt, \n",
    "                max_length=40,\n",
    "                temperature=0.8\n",
    "            )\n",
    "            print(f\"\\nLSTM Result: {lstm_result}\")\n",
    "        \n",
    "        if model_choice in ['2', '3']:\n",
    "            gpt_result = gpt_generator.generate_text(\n",
    "                prompt=user_prompt,\n",
    "                max_length=60,\n",
    "                temperature=0.8\n",
    "            )[0]\n",
    "            print(f\"\\nGPT Result: {gpt_result}\")\n",
    "\n",
    "# Uncomment the line below to run interactive generation\n",
    "# interactive_text_generation()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}